In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:

word_list = ["wizard", "university", "mountain", "lake"]

# Find similar words for each word in the list
similar_words = {word: glove_wiki_vectors.most_similar(word, topn=5) for word in word_list}
print(similar_words)

{'wizard': [('magician', 0.6618205308914185), ('sorcerer', 0.624195396900177), ('magical', 0.5892933011054993), ('magic', 0.5823098421096802), ('potter', 0.5773786306381226)], 'university': [('college', 0.8294212818145752), ('harvard', 0.8156032562255859), ('yale', 0.8113802075386047), ('professor', 0.8103784918785095), ('graduate', 0.7993001341819763)], 'mountain': [('mountains', 0.8289848566055298), ('hills', 0.7343689799308777), ('valley', 0.726043164730072), ('ridge', 0.7184715867042542), ('alpine', 0.7080119848251343)], 'lake': [('lakes', 0.7648407816886902), ('pond', 0.7507768273353577), ('river', 0.7425530552864075), ('creek', 0.721605122089386), ('springs', 0.6987648010253906)]}


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [7]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [8]:
# adapted from lec 18
for pair in word_pairs:
    print(
        "The similarity between %s and %s is %0.3f"
        % (pair[0], pair[1], glove_wiki_vectors.similarity(pair[0], pair[1]))
    )

The similarity between coast and shore is 0.700
The similarity between clothes and closet is 0.546
The similarity between old and new is 0.643
The similarity between smart and intelligent is 0.755
The similarity between dog and cat is 0.880
The similarity between tree and lawyer is 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [9]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [10]:
# Initialize an empty dictionary to store the results
word_present = {}

# Loop through each word in the test list
for word in test_words:
    # Check if the word is in the GloVe vocabulary
    is_in_vocab = word in glove_wiki_vectors
    
    # Store the result in the dictionary
    word_present[word] = is_in_vocab

# Print the results
for word, is_present in word_present.items():
    print(f"'{word}': {'True' if is_present else 'False'}")


'covididiot': False
'fomo': False
'frenemies': True
'anthropause': False
'photobomb': False
'selfie': False
'pxg': False
'pacg': False
'cct': True
'escc': True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [13]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283244

In [14]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [15]:
analogy("europe", "wealthy", "africa")

europe : wealthy :: africa : ?


,Analogy word,Score
0,impoverished,0.564562
1,affluent,0.564492
2,african,0.562155
3,wealthiest,0.532515
4,rich,0.529712
5,educated,0.519675
6,disadvantaged,0.514874
7,landless,0.506643
8,businessman,0.495366
9,young,0.495113


In [16]:
analogy("boomer", "hardworking", "millennial")

boomer : hardworking :: millennial : ?


,Analogy word,Score
0,industrious,0.540804
1,persevering,0.524354
2,hard-working,0.523273
3,virtuous,0.518251
4,resourceful,0.499835
5,tireless,0.496806
6,upstanding,0.494059
7,clannish,0.493908
8,beneficent,0.491608
9,kind-hearted,0.491580


In [17]:
analogy("slim", "desirable", "obese")

slim : desirable :: obese : ?


,Analogy word,Score
0,overweight,0.635375
1,undesirable,0.622615
2,adolescents,0.536479
3,behaviors,0.528812
4,unhealthy,0.528725
5,sedentary,0.520528
6,allergic,0.519881
7,unattractive,0.517480
8,harmful,0.510806
9,foreign-born,0.508252


In [18]:
analogy("rich", "educated", "poor")

rich : educated :: poor : ?


,Analogy word,Score
0,pupils,0.557099
1,elderly,0.554297
2,uneducated,0.551720
3,schooling,0.544815
4,students,0.540730
5,illiterate,0.537801
6,unemployed,0.534269
7,teachers,0.525769
8,graduates,0.522819
9,indifferent,0.521687


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

1) Word embeddings associating "obese" with terms like "undesirable" can reinforce harmful societal narratives and contribute to negative self-image, body shaming, and even mental health challenges. Instead, it would be better if obese was just linked to neutral medical terms like BMI,health risk etc. Focus should be on being healthy, not subjective attractiveness.

2) Embeddings with biases in real-world applications can perpetuate and amplify harmful stereotypes, discrimination, and inequality. An example would be in 2018, Amazon discovered that their AI recruiting tool was biased against women. The tool was trained on resumes submitted to the company over a decade, where most successful candidates were men. Consequently, the AI penalized resumes that included term "women" 

https://www.bbc.com/news/technology-45809919

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [21]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [22]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [23]:
import re


# Regex for initial text cleaning
def clean_text_with_regex(text):
    """
    Cleans text by removing unwanted characters and normalizing spacing.

    Parameters
    ----------
    text : str
        The raw input text to clean.

    Returns
    -------
    str
        Cleaned text.
    """
    text = re.sub(r"\n", " ", text)  # Replace newline characters with spaces
    text = re.sub(r"---+", " ", text)  # Remove repeated dashes
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    return text.strip()  # Trim leading and trailing spaces

# Preprocessing function
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Processes a spaCy doc object to clean and tokenize text.

    Parameters
    ----------
    doc : spacy.tokens.doc.Doc
        The spaCy doc object of the text.
    min_token_len : int, optional
        Minimum token length required (default is 2).
    irrelevant_pos : list, optional
        List of irrelevant POS tags to exclude (default includes punctuation, stopwords, etc.).

    Returns
    -------
    str
        The preprocessed text as a single string.
    """
    clean_text = []

    for token in doc:
        if (
            not token.is_stop  # Exclude stopwords
            and len(token) > min_token_len  # Exclude tokens below the minimum length
            and token.pos_ not in irrelevant_pos  # Exclude irrelevant POS tags
        ):
            # Replace specific token types with generic markers
            if token.like_url:
                clean_text.append("URL")
            elif token.like_email:
                clean_text.append("EMAIL")
            elif token.like_num:
                clean_text.append("NUM")
            elif token.is_alpha:  # Include only alphabetic tokens
                clean_text.append(token.lemma_.lower())  # Use lowercase lemma

    return " ".join(clean_text)

# Function to preprocess the DataFrame
def preprocess_dataframe(df, text_column="text", output_column="text_pp"):
    """
    Preprocesses the text column in a dataframe and adds a new processed text column.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the text data.
    text_column : str, optional
        The name of the column with raw text (default is "text").
    output_column : str, optional
        The name of the column to store preprocessed text (default is "text_pp").

    Returns
    -------
    pandas.DataFrame
        The updated DataFrame with a new column for preprocessed text.
    """
    # Step 1: Clean text with regex
    df[text_column] = df[text_column].apply(clean_text_with_regex)

    # Step 2: Apply spaCy preprocessing
    df[output_column] = df[text_column].apply(lambda x: preprocess(nlp(x)))

    return df


In [24]:

df = preprocess_dataframe(df)
print(df.head())

                                                text  target  \
0  You know, I was reading 18 U.S.C. 922 and some...       6   
1  It's not a bad question: I don't have any refs...       1   
2  Actuallay I don't, but on the other hand I don...       1   
3  The following problem is really bugging me, an...       2   
4  This is the latest from UPI Foreign Ministry s...       7   

             target_name                                            text_pp  
0     talk.politics.guns  know read NUM sence wonder help NUM provide pa...  
1          comp.graphics  bad question ref list algorithm think bit hard...  
2          comp.graphics  actuallay hand support idea have newsgroup asp...  
3         comp.windows.x  follow problem bug appreciate help create wind...  
4  talk.politics.mideast  late upi foreign ministry spokesman ferhat ata...  


In [25]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"Actuallay I don't, but on the other hand I don...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me, an...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,This is the latest from UPI Foreign Ministry s...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi, I'd like to subscribe to Leadership Magazi...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

1) Clean Text with Regex
Justification: These steps ensure that the text is clean and uniformly formatted, which simplifies further processing.

2) Tokenize and Process with spaCy:
Justification: Tokenization is a fundamental step to break down text into smaller, manageable units for analysis.

3) Filter Tokens Based on Characteristics:
Justification: Filtering tokens ensures that only meaningful and informative words are retained, improving the quality of the processed text.

4) Replace Specific Token Types with Placeholders:
Justification: This step standardizes representations of these token types, preserving their 
semantic relevance without allowing variability to affect the results.

5) Lemmatize Tokens:
Justification: Lemmatization helps consolidate similar words into a single form, reducing vocabulary size and improving model performance.

6) Combine Processed Tokens:
Justification: Vectorizers like CountVectorizer require input as strings rather than token lists.

7) Create a New Column in the DataFrame:
Justification: Keeping the original text alongside the preprocessed version allows for easy comparison and debugging.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

from sklearn.decomposition import LatentDirichletAllocation

n_topics = 7
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="online", max_iter=20, random_state=42
)
document_topics = lda.fit_transform(X)


print("lda.components_.shape: {}".format(lda.components_.shape))


lda.components_.shape: (7, 29284)


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [27]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

import mglearn

mglearn.tools.print_topics(
    topics=range(7),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
num           num           num           think         weaver        
url           say           game          good          tank          
file          armenian      team          player        san           
email         people        play          know          den           
program       armenians     year          let           adl           
use           turkish       win           guy           bullock       
image         come          season        time          springfield   
window        kill          player        look          utica         
available     woman         league        come          providence    
server        turkey        good          email         adirondack    


topic 5       topic 6       
--------      --------      
num           god           
gun           think         
state         know          
i

- topic 0 -> General discussion
- topic 1 -> News
- topic 2 -> Computing
- topic 3 -> Hockey
- topic 4 -> Software
- topic 5 -> History
- topic 6 -> Religion


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [28]:


# Assuming `document_topics` is the output of lda.fit_transform(X)
document_topics_df = pd.DataFrame(document_topics, columns=[f"Topic {i}" for i in range(document_topics.shape[1])])

# Add the document-topic distribution to the original DataFrame
df_with_topics = pd.concat([df, document_topics_df], axis=1)

# Show the topic assignment for the first five documents
first_five_docs = df_with_topics.loc[:, ["text_pp"] + [f"Topic {i}" for i in range(document_topics.shape[1])]].head()
print(first_five_docs)


                                             text_pp   Topic 0   Topic 1  \
0  know read NUM sence wonder help NUM provide pa...  0.002655  0.056560   
1  bad question ref list algorithm think bit hard...  0.686912  0.002018   
2  actuallay hand support idea have newsgroup asp...  0.191110  0.002751   
3  follow problem bug appreciate help create wind...  0.566552  0.006233   
4  late upi foreign ministry spokesman ferhat ata...  0.003438  0.404784   

    Topic 2   Topic 3   Topic 4   Topic 5   Topic 6  
0  0.002657  0.002651  0.002646  0.710505  0.222327  
1  0.002019  0.002018  0.002012  0.048798  0.256223  
2  0.002752  0.002754  0.002750  0.002757  0.795125  
3  0.006212  0.006215  0.006211  0.006226  0.402351  
4  0.003404  0.003403  0.003401  0.578151  0.003417  


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1 - Content-based filtering recommends items by analyzing the features of items the user has previously interacted with and matching them with the features of new items. It uses techniques like similarity measures or machine learning models to find items similar to the user's preferences, focusing solely on the user's historical data without considering other users.

2 - Filter Bubbles: Recommender systems can reinforce existing preferences, exposing users only to information or content similar to what they’ve interacted with, limiting diversity and creating echo chambers.
Privacy Concerns: Collecting and analyzing user data to make recommendations can compromise user privacy, especially if the system uses sensitive information without adequate safeguards.

3 - Transfer learning in NLP involves pre-training a model on a large corpus of general text data and then fine-tuning it on a specific task. This approach leverages the pre-trained model's knowledge of language structures and semantics, reducing the need for large task-specific labeled datasets.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)